In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/colab_notebooks/nodeNcode/socialGraph/

/content/drive/MyDrive/colab_notebooks/nodeNcode/socialGraph


In [3]:
!pip install fastcore -Uqq

In [4]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import threading
import pandas as pd
from fastcore.all import *

In [5]:
def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup


def get_main_topics(soup):
    tags = {}
    for c in soup.find_all('td'):
      tags[c.text] = c.a['href']

    return tags


def get_messages_url(topic_soup):
    message = []
    for e in topic_soup.find_all('li',attrs={'class':'disc-smr cardb'}):
        message.append(e.a['href'])
    return message


def get_message(mess):
    try:
        mess_soup = get_soup(main_url+mess)
        txt = mess_soup.find_all('div', attrs={'id':'post_content'})[0].text
        return txt, topic[0]

    except:
        #logs.append((topic[0], topic_pages_url))
        pass

In [ ]:
pagespertopic = 4
main_url = 'https://patient.info'


for char in ['e', 'f', 'g', 'h']:
    pairs = []
    logs = []
    print(f'Scrapping Character : {char}')
    url = f'https://patient.info/forums/index-{char}'

    soup = get_soup(url)
    main_topics = get_main_topics(soup)
    print(f'Number of topics in {char}: {len(main_topics)}')
    pbar = tqdm(main_topics.items(), position=0, leave=True)
    for topic in pbar:

        for i in range(1,pagespertopic+1):    
            topic_pages_url = f'?page={i}#group-discussions'
            topic_soup = get_soup(main_url+topic[1]+topic_pages_url)
            message = get_messages_url(topic_soup=topic_soup)
            
            r = parallel(get_message, message, n_workers=4)
            [pairs.append(e) for e in r]
        pbar.set_description(topic[0])

    pd.DataFrame(pairs, columns=['text', 'topic']).to_csv(f'socgraph_{char}.csv')
    #pd.DataFrame(pairs, columns=['text', 'topic']).to_csv(f'socgraph_{char}.csv')

Scrapping Character : e
Number of topics in e: 55


Ezetimibe: 100%|██████████| 55/55 [29:00<00:00, 31.65s/it]


Scrapping Character : f
Number of topics in f: 58


Flat Feet:  38%|███▊      | 22/58 [13:26<30:43, 51.20s/it]